<a href="https://colab.research.google.com/github/KyeoRaeHan/NLP/blob/main/sentence_generation/generation_deploy_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 483 kB/s 
     |████████████████████████████████| 57 kB 4.3 MB/s 
     |████████████████████████████████| 2.8 MB 51.0 MB/s 
     |████████████████████████████████| 806 kB 50.4 MB/s 
     |████████████████████████████████| 829 kB 29.2 MB/s 
     |████████████████████████████████| 134 kB 57.7 MB/s 
     |████████████████████████████████| 636 kB 40.2 MB/s 
     |████████████████████████████████| 397 kB 45.1 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 67 kB 4.0 MB/s 
     |████████████████████████████████| 3.3 MB 38.9 MB/s 
     |████████████████████████████████| 1.1 MB 27.9 MB/s 
     |████████████████████████████████| 96 kB 4.9 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
     |████████████████████████████████| 144 kB 56.1 MB/s 
     |████████████████████████████████| 271 kB 65.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

# 모델 로딩
프리트레인한 GPT2 모델과 토크나이저를 읽어 들입니다.

In [2]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained( #프리트레인을 마친 모델 생성
    "skt/kogpt2-base-v2", 
)
model.eval() #드롭아웃 등 학습 때만 필요한 기능들을 꺼서 평가모드로 동작하도록 해줌.

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [3]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


## 프롬프트 준비

언어모델에 넣을 프롬프트를 준비합니다.

In [5]:
input_ids = tokenizer.encode("안녕하세요", return_tensors="pt") #입력문장을 토큰화한 뒤 정수로 인덱싱하는 역할 수행, "pt": 인덱싱 결과를 파이토치의 텐서 자료형으로 변환

In [6]:
input_ids #'안녕하세요' 파이토치 텐서로 변환

tensor([[25906,  8702,  7801,  8084]])

In [7]:
input_ids2 = tokenizer.encode("안녕히계세요", return_tensors="pt") #입력문장을 토큰화한 뒤 정수로 인덱싱하는 역할 수행, "pt": 인덱싱 결과를 파이토치의 텐서 자료형으로 변환

In [8]:
input_ids2

tensor([[25906,  8811,  6886,  7801,  8084]])

## Greedy Search


다음 단어 확률 분포에서 최대 확률을 내는 단어들을 리턴합니다. 
여러 번 수행하더라도 생성 결과가 바뀌지 않습니다 (`do_sample=False`).
`max_length`는 생성 최대 길이이며 이보다 길거나, 짧더라도 EOD 토큰 등 스페셜 토큰이 나타나면 생성을 중단합니다. `min_length`는 생성 최소 길이이며 이보다 짧은 구간에서 EOD 등 스페셜 토큰이 등장해 생성이 중단될 경우 해당 토큰이 나올 확률을 0으로 수정하여 문장 생성이 종료되지 않도록 강제합니다.

In [9]:
import torch
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False, #확률값이 높은 단어를 다음 단어로 결정
        min_length=10,
        max_length=50,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]])) #generated_ids는 토큰 인덱스 시퀀스, decoding을 통해 자연어 문장으로 변환

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## Beam Search 

Beam Search는 다음 단어 확률 분포에서 `num_beams`만큼의 경우의 수를 남겨가면서 문장을 생성합니다. Beam search는 Greedy search보다 계산량이 많지만 좀 더 확률값이 높은 문장을 생성할 수 있습니다.

In [10]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False, #확률값이 높은 단어를 다음 단어로 결정
        min_length=10,
        max_length=50,
        num_beams=3, #빔크기, num_beams=1이면 그리디서치와 동일
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그


`num_beams=1`로 설정하면 정확히 Greedy search와 동일하게 작동합니다.


In [11]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        num_beams=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## 반복 줄이기

### 반복되는 n-gram 사이즈를 지정하기

위의 예시를 보면 `"그럼, 그건 뭐예요?"`이 반복됩니다. 이를 아래와 같이 지정해 반복을 방지합니다. 3개 이상의 토큰이 반복될 경우 해당 3-gram 등장 확률을 0으로 만들어 생성 결과에서 배제합니다.


In [13]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        no_repeat_ngram_size=3, 
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?" 하고 나는 물었다.
"그건 뭐죠?" 나는 물었다.
나는 대답하지 않았다.
"그런데 왜 그걸 물어요? 그건 무슨 뜻이에요?


### repetition penalty

repetition penalty로 반복을 통제할 수도 있습니다. 다음과 같이 실행하면 되며 그 범위는 1 이상의 값을 가져야 합니다. 1이라면 아무런 패널티를 적용하지 않는게 됩니다.


In [14]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



`repetition_penalty` 값이 클 수록 패널티가 세게 적용됩니다.

In [17]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.1, #1.0은 default, 1.0보다 클수록 페널티가 세게 적용
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"아니요, 저는요."
"그럼, 그건 무슨 말씀이신지요?"
"그럼, 그건 뭐예요?"



In [15]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.2,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요."
"그래서 오늘은 제가 할 수 있는 일이 무엇인지 말해 보겠습니다."
"이제


In [18]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요. 그리고 제가 할 수 있는 일은 아무것도 없어요.
이제 그만 돌아가고 싶어요.
제가 하는 일이 무엇


## top-k sampling

지금까지는 생성을 반복하더라도 그 결과가 동일한 샘플링 방식을 살펴봤습니다. top-k sampling은 다음 단어를 뽑을 때 확률값 기준 가장 큰 k개 가운데 하나를 선택하는 기법입니다. 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, k개 안에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있습니다. 따라서 top-k sampling은 매 시행 때마다 생성 결과가 달라집니다. k는 1 이상의 값을 지녀야 합니다.

In [19]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"라는 주제로 2시간 가량 강연했다.
이번 강연이 끝나고 질의응답이 이어지면서 강연자는 "최저임금도 그렇고 최저임금 인상 역시 그렇고, 고용불안 문제와 관련해서도 그렇고, 최저임금 인상 문제에


k=1일 경우 Greedy search와 동일합니다. 

In [20]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## top-k sampling + temperature scaling

top-k sampling은 temperature scaling과 동시에 적용할 수 있습니다. 그 값에 따라 다음과 같은 효과가 납니다.

(1) t가 0에 가까워질 수록 토큰 분포가 sharp해진다 > 1등 토큰이 뽑힐 확률이 그만큼 높아진다 > do_sample=True이지만 사실상 greedy decoding이 된다

In [21]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



(2) t=1이라면 모델 출력 분포를 그대로 사용한다 > 하지만 샘플링 방식을 사용하기 때문에 생성할 때마다 다른 문장이 나온다

In [22]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?" 하고 말했다.
이 화가는 이 작품의 여주인공인 장미의 이름이 자신의 이름에 나오는 '사랑'이 아니라 '사랑'이라 했다.
즉 장미가 사랑하는 것은 '사랑'인데, 그녀의 이름은 사랑인 것이다.
'사랑'을 '사랑하는


(3) t를 키울수록 토큰 분포가 uniform해진다 > 사실상 uniform sampling이 된다, 생성 품질이 악화할 가능성이 높아진다

In [23]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=100000000.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요’라고 반격도 나왔을 때 정 변호사와 나 교수는 전혀 상의할 문제가 없다고 일관했을 가능성이 있는 것”이란 관측부터 제기하지 않은 배경이나 나 교수에 대하고 말을 아꿨을 것이라고 분석된 셈이다.
안 원내원


## top-p sampling

top-p sampling은 다음 단어를 뽑을 때 누적 확률값이 p 이하인 단어들 가운데 하나를 선택하는 기법입니다. 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, 누적 확률값 p 이하에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있습니다. 따라서 top-p sampling은 매 시행 때마다 생성 결과가 달라집니다. p는 확률이기 때문에 0~1 사이의 값을 지녀야 합니다. p가 1이라면 어휘 집합에 있는 모든 단어를 대상으로 샘플링하기 때문에 top-p sampling 효과가 사라집니다. p가 1보다 약간 작다면 확률값이 낮은 일부 단어들을 다음 단어 후보에서 제거해 생성 품질을 높입니다. 

In [24]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?" 하며 말문이 막혔다.
안심할 수 있는 여유가 생긴 것 같았다.
그가 잠시 머뭇거리다 "감사합니다. 제가 더 노력해서 보답하겠습니다. 더 고생하겠습니다."라고 말했을 때 그녀는 이미 내심


p가 0에 가까울 경우 Greedy search와 비슷해 집니다.

In [25]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## 통합 적용

저희가 실습에 사용하고 있는 허깅페이스(huggingface) 라이브러리의 구현상 적용 순서는 다음과 같습니다.

- _get_logits_processor
  - RepetitionPenalty
  - NoRepeatNGramLogits
  - MinLengthLogits
- _get_logits_warper
  - TemperatureLogits
  - TopKLogits
  - TopPLogits

유효한 설정들을 종합 적용해 문장을 생성하는 코드는 다음과 같습니다. 샘플링(top-k, top-p)을 적용하기 때문에 시행 때마다 다른 문장이 생성됩니다.

In [26]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3,
        temperature=0.9,
        top_k=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요' 라는 글과 함께 사진이 게재되었다.
공개된 이 사진을 접한 네티즌들은 "#사진공유 <unk>
사진과 인스타그램의 차이는 확실히 있습니다.
인테리어적인 부분 뿐 아니라 디자인도 너무 고급
